In [ ]:
import torch

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import requests
import zipfile
from pathlib import Path


In [ ]:
import urllib.request
import tarfile
import os

# Download
url = "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"
filename = "food-101.tar.gz"

if not os.path.exists("food-101.tar.gz"):
    print("Downloading Food-101...")
    urllib.request.urlretrieve(url, filename)

# Extract
if not os.path.exists("food-101"):
    print("Extracting...")
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall()


In [ ]:
import os
import urllib.request
import tarfile

# Clean any broken/partial files
!rm -rf food-101 food-101.tar.gz food101_split

# Download the dataset again
url = "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"
filename = "food-101.tar.gz"

print("Downloading Food-101...")
urllib.request.urlretrieve(url, filename)

# Extract
print("Extracting...")
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall()

# Confirm meta files exist
assert os.path.exists("food-101/meta/train.txt"), "train.txt not found!"
assert os.path.exists("food-101/meta/test.txt"), "test.txt not found!"
print("✅ Dataset is downloaded and extracted properly.")


In [ ]:
import shutil
from tqdm import tqdm

def copy_images(split):
    with open(f"food-101/meta/{split}.txt", "r") as f:
        lines = f.read().splitlines()

    for line in tqdm(lines, desc=f"Copying {split} data"):
        class_name, image_name = line.split("/")
        src = f"food-101/images/{class_name}/{image_name}.jpg"
        dst_dir = f"food101_split/{split}/{class_name}/"
        dst = os.path.join(dst_dir, f"{image_name}.jpg")

        os.makedirs(dst_dir, exist_ok=True)
        shutil.copyfile(src, dst)

os.makedirs("food101_split", exist_ok=True)
copy_images("train")
copy_images("test")


In [18]:
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torchvision.datasets import ImageFolder

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder(root = '/content/food101_split/train',transform = transform)
test_dataset = datasets.ImageFolder(root = '/content/food101_split/test',transform = transform)
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [19]:
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model  = models.efficientnet_b0(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features,num_classes)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(),lr = 0.001)
model = model.to(device)


In [20]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)


In [21]:
def evaluate_model(model:nn.Module,dataloader:torch.utils.data.DataLoader):
  model.eval()
  correct = 0
  total = 0

  with torch.inference_mode():
    for images,labels in test_loader:
      images,labels = images.to(device),labels.to(device)
      model = model.to(device)
      preds = model(images).argmax(dim=1)
      correct += (labels == preds).sum().item()
      total = labels.size(0)

  accuracy = correct/total
  print(f"Accuracy: {accuracy: .3f}")

  return accuracy

In [22]:
def train_test(model: nn.Module,
               previous_accuracy: float,
               previous_loss: float,
               epoch: int,
               train_loader: torch.utils.data.DataLoader,
               test_loader: torch.utils.data.DataLoader):

    model = model.to(device)
    model.train()
    total_loss = 0
    losses_count = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss_value = loss(output, labels)
        loss_value.backward()
        optimizer.step()

        total_loss += loss_value.item()
        losses_count += 1

    average_loss_pe = total_loss / losses_count
    current_loss = average_loss_pe

    print(f"Epoch {epoch+1}, Loss: {average_loss_pe:.4f}")

    if current_loss >= previous_loss:
        print("The model is starting to overfit (loss increasing). Stopping training.")
        return

    current_accuracy = evaluate_model(model, test_loader)

    if current_accuracy <= previous_accuracy:
        print("The model is starting to overfit (accuracy dropping). Stopping training.")
        return

    if epoch >= 20:
        print("More than 20 epochs done. Stopping.")
        return

    train_test(model, current_accuracy, current_loss, epoch + 1, train_loader, test_loader)


In [23]:
import torchvision.models as models
import torch.nn as nn

custom_classifier = model.classifier

model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

model.classifier = custom_classifier

model = model.to(device)
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()



In [24]:
train_test(model,evaluate_model(model,test_loader),float("inf"),0,train_loader,test_loader)

KeyboardInterrupt: 

In [25]:
if device == 'cuda':
  print("Cude is available ")
else :
  print("Cpu")

Cpu


In [33]:
if device == torch.device('cuda'):
    print("Device is CUDA")
else:
    print("Device is not CUDA")


Device is not CUDA
